In [ ]:
# !pip install dash

In [ ]:
import pandas as pd
import plotly.express as px
import os
import json
import re
import numpy as np
import plotly.express as px
import json
from datetime import datetime

# Fetch the data

## Disaster Data

In [ ]:
df = pd.read_excel('Data Bencana.xlsx', header=1)

In [ ]:
df.head()

,No.,Kode Identitas Bencana,ID Kabupaten,Tanggal Kejadian,Kejadian,Lokasi,Kabupaten,Provinsi,Kronologi & Dokumentasi,Penyebab,Meninggal,Hilang,Terluka,Rumah Rusak,Rumah Terendam,Fasum Rusak
0,1,NaN,6202,2025-12-04,BANJIR,Kecamatan:\nKec. Tualan Hulu Ds. Tumbang Mujam,Kotawaringin Timur,Kalimantan Tengah,Dokumentasi,NaN,NaN,NaN,NaN,0.0,NaN,0
1,2,NaN,3313,2025-12-04,TANAH LONGSOR,Kecamatan:\nKec. Jenawi,Karanganyar,Jawa Tengah,Dokumentasi,NaN,NaN,NaN,NaN,0.0,NaN,0
2,3,NaN,3213,2025-12-04,BANJIR,Kecamatan:\nKec. Cisalak,Subang,Jawa Barat,Dokumentasi,NaN,NaN,NaN,NaN,64.0,NaN,0
3,4,NaN,3213,2025-12-04,BANJIR,Kecamatan:\nKec. Legon Kulo Ds. Mayangan Ds. ...,Subang,Jawa Barat,Dokumentasi,NaN,NaN,NaN,NaN,0.0,NaN,0
4,5,NaN,3212,2025-12-04,BANJIR,Kecamatan:\nKec. Kandanghaur Ds. Eretan Kulon,Indramayu,Jawa Barat,Dokumentasi,NaN,NaN,NaN,NaN,0.0,NaN,0


In [ ]:
df.isna().sum()

,0
No.,0
Kode Identitas Bencana,1527
ID Kabupaten,0
Tanggal Kejadian,0
Kejadian,0
Lokasi,924
Kabupaten,0
Provinsi,0
Kronologi & Dokumentasi,0
Penyebab,11585


In [ ]:
df['Kejadian'].value_counts()

,count
Kejadian,
BANJIR,9447
CUACA EKSTREM,7205
KEBAKARAN HUTAN DAN LAHAN,5280
TANAH LONGSOR,4592
KEKERINGAN,359
GEMPABUMI,303
GELOMBANG PASANG DAN ABRASI,245
ERUPSI GUNUNG API,33
TSUNAMI,2


## GeoJSON Data

In [ ]:
# use github repo instead
link = 'https://github.com/JfrAziz/indonesia-district'
repo_name = link.split("/")[-1]
!rm -rf {repo_name}
!git clone {link}

Cloning into 'indonesia-district'...
remote: Enumerating objects: 8195, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8195 (delta 4), reused 9 (delta 4), pack-reused 8184 (from 1)
Receiving objects: 100% (8195/8195), 411.07 MiB | 16.11 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Updating files: 100% (7632/7632), done.


In [ ]:
repo_name = link.split("/")[-1]
!ls -R /content/{repo_name}

Streaming output truncated to the last 5000 lines.
id1212_deli_serdang.geojson

/content/indonesia-district/id12_sumatera_utara/id1213_langkat:
id1213010_bohorok.geojson	   id1213110_padang_tualang.geojson
id1213011_sirapit.geojson	   id1213120_hinai.geojson
id1213020_salapian.geojson	   id1213130_secanggang.geojson
id1213021_kutambaru.geojson	   id1213140_tanjung_pura.geojson
id1213030_sei_bingai.geojson	   id1213150_gebang.geojson
id1213040_kuala.geojson		   id1213160_babalan.geojson
id1213050_selesai.geojson	   id1213170_sei_lepan.geojson
id1213060_binjai.geojson	   id1213180_brandan_barat.geojson
id1213070_stabat.geojson	   id1213190_besitang.geojson
id1213080_wampu.geojson		   id1213200_pangkalan_susu.geojson
id1213090_batang_serangan.geojson  id1213201_pematang_jaya.geojson
id1213100_sawit_seberang.geojson   id1213_langkat.geojson

/content/indonesia-district/id12_sumatera_utara/id1214_nias_selatan:
id1214010_hibala.geojson		  id1214051_siduaori.geojson
id1214011_tanah_masa.geojs

## Extract All of the Kecamatan data for later use

In [ ]:
repo_path = f'/content/{repo_name}'
set_kecamatan = []

for root, dirs, files in os.walk(repo_path):
  for file in files:
    if file.endswith('.geojson'):
      match = re.match(r'id\d+_(.+)\.geojson', file)
      if match:
        place_name = match.group(1)
        set_kecamatan.append(place_name)
set_kecamatan = list(set(set_kecamatan))


In [ ]:
print(f"Found {len(set_kecamatan)} unique places")
print("Sample places:", set_kecamatan[:10])

Found 7110 unique places
Sample places: ['tabunganen', 'kebun_tebu', 'muting', 'sumberlawang', 'batang_tuaka', 'limboto', 'kepulauan_talaud', 'teluk_ambon_baguala', 'kotamobagu_timur', 'cipondoh']


In [ ]:
set_kecamatan = [s.replace("_"," ") for s in set_kecamatan]
set_kecamatan

['tabunganen',
 'kebun tebu',
 'muting',
 'sumberlawang',
 'batang tuaka',
 'limboto',
 'kepulauan talaud',
 'teluk ambon baguala',
 'kotamobagu timur',
 'cipondoh',
 'duapitue',
 'subi',
 'tubbi taramanu',
 'kota jakarta barat',
 'andey',
 'kikim selatan',
 'abuki',
 'rumbai pesisir',
 'seluma timur',
 'ciampel',
 'compreng',
 'tareran',
 'bawolato',
 'biaro',
 'pagak',
 'anggruk',
 'poso',
 'abepura',
 'seputih raman',
 'aek kuasan',
 'manggarai barat',
 'katapang',
 'merakurak',
 'joerat',
 'nusa tabukan',
 'soppeng riaja',
 'bungur',
 'petasia timur',
 'sungai bungkal',
 'demon pagong',
 'pintu rime gayo',
 'saipar dolok hole',
 'dogiyai',
 'singingi',
 'cikarang pusat',
 'ayip',
 'kelumbayan barat',
 'gajah',
 'beber',
 'sekar',
 'balai jaya',
 'sedan',
 'terbanggi besar',
 'tenga',
 'medan baru',
 'losarang',
 'binongko',
 'jatinunggal',
 'mulak sebingkai',
 'hutan lindung',
 'serangpanjang',
 'negeri katon',
 'kota lubuklinggau',
 'tambun selatan',
 'kambrau',
 'kelumpang hilir'

## Remove prefix id in the local directory

In [ ]:
def remove_id_prefix(name):
    return re.sub(r'^id', '', name)

In [ ]:
for root, dirs, files in os.walk(repo_path,topdown=False):
    for file in files:
        if file.startswith('id'):
            old_path = os.path.join(root, file)
            new_name = remove_id_prefix(file)
            new_path = os.path.join(root, new_name)
            os.rename(old_path, new_path)
            # print(f"File: {file} → {new_name}")

    for dir_name in dirs:
        if dir_name.startswith('id'):
            old_path = os.path.join(root, dir_name)
            new_name = remove_id_prefix(dir_name)
            new_path = os.path.join(root, new_name)
            os.rename(old_path, new_path)
            # print(f"Dir: {dir_name} → {new_name}")

In [ ]:
repo_name = link.split("/")[-1]
!ls -R /content/{repo_name}

Streaming output truncated to the last 5000 lines.
1203010_batang_angkola.geojson		1203101_marancar.geojson
1203011_sayur_matinggi.geojson		1203102_muara_batang_toru.geojson
1203012_tano_tombangan_angkola.geojson	1203110_sipirok.geojson
1203070_angkola_timur.geojson		1203120_arse.geojson
1203080_angkola_selatan.geojson		1203160_saipar_dolok_hole.geojson
1203090_angkola_barat.geojson		1203161_aek_bilah.geojson
1203091_angkola_sangkunur.geojson	1203_tapanuli_selatan.geojson
1203100_batang_toru.geojson

/content/indonesia-district/12_sumatera_utara/1204_tapanuli_tengah:
1204010_pinang_sori.geojson   1204060_sorkam.geojson
1204011_badiri.geojson	      1204061_sorkam_barat.geojson
1204020_sibabangun.geojson    1204062_pasaribu_tobing.geojson
1204021_lumut.geojson	      1204070_barus.geojson
1204022_sukabangun.geojson    1204071_sosor_gadong.geojson
1204030_pandan.geojson	      1204072_andam_dewi.geojson
1204031_tukka.geojson	      1204073_barus_utara.geojson
1204032_sarudik.geojson       12

# Data Preprocessing

## Filter Only The Needed Columns

In [ ]:
not_drop = ['Kabupaten','Provinsi', 'Lokasi','Tanggal Kejadian','Kejadian','ID Kabupaten']

In [ ]:
to_drop = [col for col in df.columns if col not in not_drop]

In [ ]:
to_drop

['No.',
 'Kode Identitas Bencana',
 'Kronologi & Dokumentasi',
 'Penyebab',
 'Meninggal',
 'Hilang',
 'Terluka',
 'Rumah Rusak',
 'Rumah Terendam',
 'Fasum Rusak']

In [ ]:
df.drop(to_drop,axis=1,inplace=True)

In [ ]:
df.head()

,ID Kabupaten,Tanggal Kejadian,Kejadian,Lokasi,Kabupaten,Provinsi
0,6202,2025-12-04,BANJIR,Kecamatan:\nKec. Tualan Hulu Ds. Tumbang Mujam,Kotawaringin Timur,Kalimantan Tengah
1,3313,2025-12-04,TANAH LONGSOR,Kecamatan:\nKec. Jenawi,Karanganyar,Jawa Tengah
2,3213,2025-12-04,BANJIR,Kecamatan:\nKec. Cisalak,Subang,Jawa Barat
3,3213,2025-12-04,BANJIR,Kecamatan:\nKec. Legon Kulo Ds. Mayangan Ds. ...,Subang,Jawa Barat
4,3212,2025-12-04,BANJIR,Kecamatan:\nKec. Kandanghaur Ds. Eretan Kulon,Indramayu,Jawa Barat


## Add Missing ID's

### Extract all mapping of ids from the repo

In [ ]:
province_name_to_id = {}
province_id_to_name = {}
city_name_to_id = {}
city_id_to_name = {}
subdistrict_name_to_id = {}
subdistrict_id_to_name = {}

In [ ]:
for root, dirs, files in os.walk(repo_path):
    rel_path = os.path.relpath(root, repo_path)
    path_parts = rel_path.split(os.sep)

    if rel_path == '.':
        continue

    if len(path_parts) == 1:
        province = path_parts[0]
        if '_' in province:
            prov_id, prov_name = province.split('_', 1)
            province_name_to_id[prov_name] = prov_id
            province_id_to_name[prov_id] = prov_name

    elif len(path_parts) == 2:
        province = path_parts[0]
        city = path_parts[1]

        if '_' in city:
            city_id, city_name = city.split('_', 1)
            city_name_to_id[city_name] = city_id
            city_id_to_name[city_id] = city_name

    for file in files:
        if file.endswith('.geojson') and '_' in file:
            name = file.replace('.geojson', '')
            if '_' in name:
                subdistrict_id, subdistrict_name = name.split('_', 1)

                if len(subdistrict_id) > 4:
                    subdistrict_name_to_id[subdistrict_name] = subdistrict_id
                    subdistrict_id_to_name[subdistrict_id] = subdistrict_name

In [ ]:
print(f"Provinces: {len(province_name_to_id)}")
print(f"Cities: {len(city_name_to_id)}")
print(f"Subdistricts: {len(subdistrict_name_to_id)}")


Provinces: 34
Cities: 519
Subdistricts: 6679


### Add Missing Province ID

In [ ]:
df['Provinsi'].unique()

array(['Kalimantan Tengah', 'Jawa Tengah', 'Jawa Barat', 'Banten',
       'Sulawesi Selatan', 'Jawa Timur', 'Nusa Tenggara Barat', 'Lampung',
       'Gorontalo', 'Maluku Utara', 'Sulawesi Barat', 'Sumatera Barat',
       'Sumatera Utara', 'Aceh', 'Riau', 'Daerah Istimewa Yogyakarta',
       'Sulawesi Tenggara', 'Sulawesi Tengah', 'Sulawesi Utara',
       'Kalimantan Utara', 'Kalimantan Selatan', 'Maluku',
       'Kalimantan Barat', 'Kalimantan Timur', 'Nusa Tenggara Timur',
       'Sumatera Selatan', 'Papua Pegunungan', 'DKI Jakarta',
       'Kepulauan Riau', 'Bengkulu', 'Papua', 'Papua Tengah', 'Jambi',
       'Bali', 'Papua Barat Daya', 'Kepulauan Bangka Belitung',
       'Papua Selatan', 'Papua Barat', 'NUSA TENGGARA BARAT',
       'JAWA TIMUR', 'RIAU', 'SUMATERA UTARA', 'KALIMANTAN SELATAN',
       'SULAWESI UTARA', 'JAWA TENGAH', 'JAWA BARAT', 'ACEH',
       'SULAWESI TENGAH', 'KEPULAUAN RIAU', 'NUSA TENGGARA TIMUR',
       'MALUKU', 'DAERAH ISTIMEWA YOGYAKARTA', 'LAMPUNG', 'BANTE

In [ ]:
df.loc[df['Provinsi'] == 'P A P U A', 'Provinsi'] = 'PAPUA'

In [ ]:
df.loc[:,'Provinsi'] = df['Provinsi'].str.lower()

In [ ]:
df.loc[:,'Provinsi'] = df['Provinsi'].str.replace(" ","_")


In [ ]:
province_name_to_id

{'bengkulu': '17',
 'kalimantan_tengah': '62',
 'jambi': '15',
 'lampung': '18',
 'bali': '51',
 'nusa_tenggara_timur': '53',
 'jawa_tengah': '33',
 'sumatera_utara': '12',
 'jawa_barat': '32',
 'sulawesi_utara': '71',
 'papua_barat': '91',
 'kalimantan_timur': '64',
 'sulawesi_barat': '76',
 'sumatera_barat': '13',
 'sulawesi_selatan': '73',
 'kalimantan_barat': '61',
 'gorontalo': '75',
 'papua': '94',
 'kalimantan_utara': '65',
 'dki_jakarta': '31',
 'maluku': '81',
 'sumatera_selatan': '16',
 'daerah_istimewa_yogyakarta': '34',
 'sulawesi_tengah': '72',
 'kepulauan_riau': '21',
 'sulawesi_tenggara': '74',
 'aceh': '11',
 'kalimantan_selatan': '63',
 'nusa_tenggara_barat': '52',
 'banten': '36',
 'jawa_timur': '35',
 'maluku_utara': '82',
 'riau': '14',
 'kepulauan_bangka_belitung': '19'}

In [ ]:
for prov in df['Provinsi'].unique():
  if not prov in province_name_to_id.keys():
    print(prov)

papua_pegunungan
papua_tengah
papua_barat_daya
papua_selatan
di_yogyakarta


In [ ]:
# drop all papua for now
df = df[~df['Provinsi'].str.contains('papua', case=False, na=False)]

In [ ]:
df.loc[df['Provinsi'] == 'di_yogyakarta','Provinsi'] = 'daerah_istimewa_yogyakarta'

In [ ]:
df['Provinsi'].unique()

array(['kalimantan_tengah', 'jawa_tengah', 'jawa_barat', 'banten',
       'sulawesi_selatan', 'jawa_timur', 'nusa_tenggara_barat', 'lampung',
       'gorontalo', 'maluku_utara', 'sulawesi_barat', 'sumatera_barat',
       'sumatera_utara', 'aceh', 'riau', 'daerah_istimewa_yogyakarta',
       'sulawesi_tenggara', 'sulawesi_tengah', 'sulawesi_utara',
       'kalimantan_utara', 'kalimantan_selatan', 'maluku',
       'kalimantan_barat', 'kalimantan_timur', 'nusa_tenggara_timur',
       'sumatera_selatan', 'dki_jakarta', 'kepulauan_riau', 'bengkulu',
       'jambi', 'bali', 'kepulauan_bangka_belitung'], dtype=object)

In [ ]:
df['ID Provinsi'] = df.loc[:,'Provinsi'].apply(lambda x : province_name_to_id[x])

### Create list of kecamatan column

In [ ]:
pd.set_option('display.max_rows', None)

# Show full column width
pd.set_option('display.max_colwidth', None)

print(df['Lokasi'].sample(n=10))

19732    Lokasi :   • 57 km Tenggara Kab. Blitar • 60 km BaratDaya KAB-MALANG-JATIM • 62 km Tenggara KOTA-BLITAR-JATIM • 159 km BaratDaya SURABAYA-JATIM • 671 km Tenggara JAKARTA-INDONESIA
8674                                                                                                                                                           Kec. Kesugihan Ds. Bulupayung
337                                                                                                                                                                 Kecamatan:\nKec. Nglegok
2571                                                                                                                                                               Kecamatan:\nKec. Ngemplak
19106                                                                                                                                      Kec. Juai, Ds. Gulinggan, Kec. Halong, Ds. Gangsa
19511                                                  

In [ ]:
kecamatan_names_normalized = {name.replace('_',' '): subdistrict_id for name, subdistrict_id in subdistrict_name_to_id.items()}
kecamatan_names_normalized

{'manna': '1701040',
 'kedurang': '1701042',
 'kedurang ilir': '1701045',
 'pino': '1701060',
 'pinoraya': '1701061',
 'bunga mas': '1701043',
 'air nipis': '1701051',
 'ulu manna': '1701062',
 'pasar manna': '1701044',
 'seginim': '1701050',
 'kota manna': '1701041',
 'ratu agung': '1771021',
 'kampung melayu': '1771011',
 'teluk segara': '1771030',
 'ratu samban': '1771022',
 'gading cempaka': '1771020',
 'singaran pati': '1771023',
 'selebar': '1771010',
 'sungai serut': '1771031',
 'muara bangka hulu': '1771040',
 'bindu riang': '1702032',
 'sindang beliti ulu': '1702033',
 'curup timur': '1702047',
 'bermani ulu raya': '1702045',
 'selupu rejang': '1702042',
 'sindang dataran': '1702034',
 'sindang kelingi': '1702031',
 'padang ulak tanding': '1702030',
 'bermani ulu': '1702041',
 'curup tengah': '1702044',
 'kota padang': '1702020',
 'curup selatan': '1702043',
 'sindang beliti ilir': '1702021',
 'curup utara': '1702046',
 'curup': '1702040',
 'muara sahung': '1704042',
 'kinal':

In [ ]:
def find_kecamatan(str_lokasi):
  if pd.isna(str_lokasi):
    return []

  str_lokasi_lower = str(str_lokasi).lower()

  matches = []
  for kec_name, kec_id in kecamatan_names_normalized.items():
    if kec_name.lower() in str_lokasi_lower:
      matches.append([kec_name,kec_id])

  return matches

In [ ]:
df['ID Kecamatan'] = df['Lokasi'].apply(find_kecamatan)

In [ ]:
df['ID Kecamatan'].head()

,ID Kecamatan
0,"[[tualan hulu, 6202201]]"
1,"[[jenawi, 3313170]]"
2,"[[alak, 5371010], [salak, 1216010], [cisalak, 3213030]]"
3,"[[pati, 3318100], [muara, 1205180], [pusakanagara, 3213200], [blanakan, 3213220], [kulo, 7314051], [mayangan, 3574030], [mayang, 3509080]]"
4,"[[kandanghaur, 3212190], [kulo, 7314051]]"


In [ ]:
df.head()

,ID Kabupaten,Tanggal Kejadian,Kejadian,Lokasi,Kabupaten,Provinsi,ID Provinsi,ID Kecamatan
0,6202,2025-12-04,BANJIR,Kecamatan:\nKec. Tualan Hulu Ds. Tumbang Mujam,Kotawaringin Timur,kalimantan_tengah,62,"[[tualan hulu, 6202201]]"
1,3313,2025-12-04,TANAH LONGSOR,Kecamatan:\nKec. Jenawi,Karanganyar,jawa_tengah,33,"[[jenawi, 3313170]]"
2,3213,2025-12-04,BANJIR,Kecamatan:\nKec. Cisalak,Subang,jawa_barat,32,"[[alak, 5371010], [salak, 1216010], [cisalak, 3213030]]"
3,3213,2025-12-04,BANJIR,Kecamatan:\nKec. Legon Kulo Ds. Mayangan Ds. Legon Kec. Pusakanagara Ds. Patimban Kec. Blanakan Ds. Muara,Subang,jawa_barat,32,"[[pati, 3318100], [muara, 1205180], [pusakanagara, 3213200], [blanakan, 3213220], [kulo, 7314051], [mayangan, 3574030], [mayang, 3509080]]"
4,3212,2025-12-04,BANJIR,Kecamatan:\nKec. Kandanghaur Ds. Eretan Kulon,Indramayu,jawa_barat,32,"[[kandanghaur, 3212190], [kulo, 7314051]]"


In [ ]:
df.drop(labels='Lokasi',inplace=True,axis=1)

In [ ]:
df.head()

,ID Kabupaten,Tanggal Kejadian,Kejadian,Kabupaten,Provinsi,ID Provinsi,ID Kecamatan
0,6202,2025-12-04,BANJIR,Kotawaringin Timur,kalimantan_tengah,62,"[[tualan hulu, 6202201]]"
1,3313,2025-12-04,TANAH LONGSOR,Karanganyar,jawa_tengah,33,"[[jenawi, 3313170]]"
2,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[[alak, 5371010], [salak, 1216010], [cisalak, 3213030]]"
3,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[[pati, 3318100], [muara, 1205180], [pusakanagara, 3213200], [blanakan, 3213220], [kulo, 7314051], [mayangan, 3574030], [mayang, 3509080]]"
4,3212,2025-12-04,BANJIR,Indramayu,jawa_barat,32,"[[kandanghaur, 3212190], [kulo, 7314051]]"


In [ ]:
df = df.explode('ID Kecamatan').reset_index(drop=True)
df.head()

,ID Kabupaten,Tanggal Kejadian,Kejadian,Kabupaten,Provinsi,ID Provinsi,ID Kecamatan
0,6202,2025-12-04,BANJIR,Kotawaringin Timur,kalimantan_tengah,62,"[tualan hulu, 6202201]"
1,3313,2025-12-04,TANAH LONGSOR,Karanganyar,jawa_tengah,33,"[jenawi, 3313170]"
2,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[alak, 5371010]"
3,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[salak, 1216010]"
4,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[cisalak, 3213030]"


In [ ]:
df.head()

,ID Kabupaten,Tanggal Kejadian,Kejadian,Kabupaten,Provinsi,ID Provinsi,ID Kecamatan
0,6202,2025-12-04,BANJIR,Kotawaringin Timur,kalimantan_tengah,62,"[tualan hulu, 6202201]"
1,3313,2025-12-04,TANAH LONGSOR,Karanganyar,jawa_tengah,33,"[jenawi, 3313170]"
2,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[alak, 5371010]"
3,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[salak, 1216010]"
4,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,"[cisalak, 3213030]"


In [ ]:
def extract_kecamatan_name(x):
    try:
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return None
        if isinstance(x, (list, np.ndarray)) and len(x) >= 2:
            return x[0]
        return None
    except:
        return None

def extract_kecamatan_id(x):
    try:
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return None
        if isinstance(x, (list, np.ndarray)) and len(x) >= 2:
            return x[1]
        return None
    except:
        return None

In [ ]:
df['Kecamatan'] = df['ID Kecamatan'].apply(extract_kecamatan_name)
df['ID Kecamatan'] = df['ID Kecamatan'].apply(extract_kecamatan_id)


In [ ]:
df = df.drop_duplicates(keep='first').reset_index(drop=True)

In [ ]:
len(df)

107111

In [ ]:
has_duplicates = df.duplicated().any()
has_duplicates

np.False_

In [ ]:
df.head()

,ID Kabupaten,Tanggal Kejadian,Kejadian,Kabupaten,Provinsi,ID Provinsi,ID Kecamatan,Kecamatan
0,6202,2025-12-04,BANJIR,Kotawaringin Timur,kalimantan_tengah,62,6202201,tualan hulu
1,3313,2025-12-04,TANAH LONGSOR,Karanganyar,jawa_tengah,33,3313170,jenawi
2,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,5371010,alak
3,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,1216010,salak
4,3213,2025-12-04,BANJIR,Subang,jawa_barat,32,3213030,cisalak


In [ ]:
df.sort_values(df.columns.tolist(), ignore_index=True).head()

,ID Kabupaten,Tanggal Kejadian,Kejadian,Kabupaten,Provinsi,ID Provinsi,ID Kecamatan,Kecamatan
0,1101,2020-01-09,BANJIR,ACEH SELATAN,aceh,11,1103010,trumon
1,1101,2020-01-09,BANJIR,ACEH SELATAN,aceh,11,1103012,trumon tengah
2,1101,2020-01-09,BANJIR,ACEH SELATAN,aceh,11,1103080,meukek
3,1101,2020-01-09,BANJIR,ACEH SELATAN,aceh,11,1115030,beutong
4,1101,2020-01-09,BANJIR,ACEH SELATAN,aceh,11,1407022,ujung batu


## Download CSV

In [ ]:
df.to_csv('granular.csv')

KeyError: 'column_name'